In [34]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_apd as dr_a
import tools.data_reader_ascertain as dr_asc
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)
warnings.simplefilter(action='ignore', category=FutureWarning)# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_ascertain as dr_asc
import tools.data_reader_wesad as dr_w
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)
warnings.simplefilter(action='ignore', category=FutureWarning)

In [79]:
metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.IBI, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE
]

# model_phases_wesad = [
#     [
        # dr_w.Phases.BASE,
#         dr_w.Phases.FUN,
#         dr_w.Phases.TSST,
#         dr_w.Phases.MEDI_1,
#         dr_w.Phases.MEDI_2
#     ],
    # [
    #     dr_w.Phases.BASE,
    #     dr_w.Phases.TSST,
    #     dr_w.Phases.MEDI_1,
    #     dr_w.Phases.MEDI_2
    # ],
    # [
    #     dr_w.Phases.BASE,
    #     dr_w.Phases.MEDI_1,
    #     dr_w.Phases.MEDI_2
    # ],
    # [
    #     dr_w.Phases.BASE,
    # ]
# ]

model_phases_wesad = dr_w.Phases.PHASE_ORDER

wesad_label_type = "stai"
asc_label_type = dr_asc.SelfReports.AROUSAL

models = {
    # "SVM": SVC(C=10, gamma=1),  # C=10, gamma=1
    # "KNN": KNeighborsClassifier(n_neighbors=7),
    # "DT": DecisionTreeClassifier(),
    # "LogReg": LogisticRegression(max_iter=1000),
    # "Bayes": GaussianNB(),
    "XGB": XGBClassifier(use_label_encoder=False, objective="binary:logistic", eval_metric="logloss")
}

threshold = "fixed"
asc_threshold = "fixed"
test_size = 1.0


In [67]:
importlib.reload(train)
importlib.reload(dr_w)

stai_scores, dim_scores_arousal, dim_scores_valence = train.Train_WESAD.get_labels(model_phases_wesad)
print(stai_scores)

    subject  Base_STAI  TSST_STAI  Medi_1_STAI   Fun_STAI  Medi_2_STAI
0         2  33.333333  66.666667    23.333333  26.666667    20.000000
1         4  30.000000  56.666667    50.000000  26.666667    43.333333
2         5  33.333333  60.000000    26.666667  40.000000    26.666667
3         6  30.000000  50.000000    30.000000  33.333333    20.000000
4         7  26.666667  60.000000    30.000000  23.333333    36.666667
5         8  43.333333  60.000000    46.666667  43.333333    40.000000
6         9  30.000000  46.666667    23.333333  33.333333    23.333333
7        10  36.666667  66.666667    33.333333  26.666667    40.000000
8        11  30.000000  63.333333    26.666667  20.000000    26.666667
9        13  46.666667  70.000000    26.666667  26.666667    53.333333
10       14  43.333333  60.000000    43.333333  40.000000    43.333333
11       15  40.000000  63.333333    30.000000  33.333333    30.000000
12       16  43.333333  63.333333    30.000000  36.666667    33.333333
13    

In [82]:
# TRAIN ON ASCERTAIN AND TEST ON WESAD
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_w)
importlib.reload(dt)


for j, phases_wesad in enumerate(model_phases_wesad):
    print(f"WESAD PHASES {phases_wesad} " + "-"*50)
    phases_wesad = [phases_wesad]
    x_a, y_a = train.Train_ASCERTAIN.get_ascertain_data(metrics, verbose=False, label_type=asc_label_type, threshold=asc_threshold)
    x_b, y_b = train.Train_WESAD.get_wesad_data(metrics, phases_wesad, verbose=False, label_type=wesad_label_type, threshold=threshold)
    # drop subjects with noisy data
    inds = pd.isnull(x_a).any(axis=1).to_numpy().nonzero()[0]
    x_a = x_a.drop(labels=inds, axis=0)
    y_a = y_a.drop(labels=inds, axis=0)
    
    # x = x[x['subject'] != 8.0]
    # y = y[y['subject'] != 8.0]
    
    x_a = x_a.drop(["phaseId"], axis=1)
    x_b = x_b.drop(["phaseId"], axis=1)

    # make sure subjects from different datasets aren't labeled with the same index
    x_b["subject"] = x_b["subject"] + 500
    y_b["subject"] = y_b["subject"] + 500

    # print(f"y_a:\n{y_a.loc[:, 'label'].value_counts()}")
    # print(f"y_b:\n{y_b.loc[:, 'label'].value_counts()}")

    # # 0-1 scaling
    # for c in range(3, len(x_a.columns)):
    #     data_col = x_a[x_a.columns[c]]
    #     data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
    #     x_a[x_a.columns[c]] = data_col
    # # 0-1 scaling
    # for c in range(3, len(x_b.columns)):
    #     data_col = x_b[x_b.columns[c]]
    #     data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
    #     x_b[x_b.columns[c]] = data_col

    acc_results = {
        # "SVM": [],
        # "LogReg": [],
        "XGB": []
    }
    reports = {
        # "SVM": [],
        # "LogReg": [],
        "XGB": []
    }
    num_iters = 10
    for _ in range(num_iters):
        try:
            out = train.Train_Multi_Dataset.train_across_datasets(models, x_a, y_a, x_b, y_b, by_subject=True, save_metrics=True, test_size=test_size, is_resample=False, get_shap_values=True)
            for model_name in acc_results:
                acc_results[model_name].append(out[model_name][0])
                reports[model_name].append(out[model_name][1])
        except Exception as e:
            print("Error in resampling train/test data")

    for model_name in acc_results.keys():
        acc = np.mean(acc_results[model_name])
        print(f"{model_name} accuracy over {num_iters} rounds: {acc}")
        if acc > 0:
            print(f"Model evaluation metrics for {model_name}:")
            p = np.mean([report["precision"] for report in reports[model_name]])
            r = np.mean([report["recall"] for report in reports[model_name]])
            f1 = np.mean([report["f1"] for report in reports[model_name]])
            auc = np.mean([report["auc"] for report in reports[model_name]])
            report = reports[model_name]
            print(f"Precision: {p}\nRecall: {r}\nF1-score: {f1}\nAUC score: {auc}")
        
    print("\n")

WESAD PHASES Base --------------------------------------------------
y_train:
1    1319
0     228
Name: label, dtype: int64
y_test:
0    14
Name: label, dtype: int64
Model XGB, Predictions: [0 1], [11  3]
Only one class present in y_true. ROC AUC score is not defined in that case. Setting AUC score to -1.
Calculating SHAP values


ntree_limit is deprecated, use `iteration_range` or model slicing instead.


y_train:
1    1319
0     228
Name: label, dtype: int64
y_test:
0    14
Name: label, dtype: int64
Model XGB, Predictions: [0 1], [11  3]
Only one class present in y_true. ROC AUC score is not defined in that case. Setting AUC score to -1.
Calculating SHAP values


ntree_limit is deprecated, use `iteration_range` or model slicing instead.


y_train:
1    1319
0     228
Name: label, dtype: int64
y_test:
0    14
Name: label, dtype: int64
Model XGB, Predictions: [0 1], [11  3]
Only one class present in y_true. ROC AUC score is not defined in that case. Setting AUC score to -1.
Calculating SHAP values


...

ntree_limit is deprecated, use `iteration_range` or model slicing instead.


y_train:
1    1319
0     228
Name: label, dtype: int64
y_test:
0    13
1     1
Name: label, dtype: int64
Model XGB, Predictions: [0 1], [ 3 11]
Calculating SHAP values


ntree_limit is deprecated, use `iteration_range` or model slicing instead.


y_train:
1    1319
0     228
Name: label, dtype: int64
y_test:
0    13
1     1
Name: label, dtype: int64
Model XGB, Predictions: [0 1], [ 3 11]
Calculating SHAP values
XGB accuracy over 10 rounds: 0.2857142857142857
Model evaluation metrics for XGB:
Precision: 0.09090909090909091
Recall: 1.0
F1-score: 0.16666666666666669
AUC score: 0.6153846153846153




ntree_limit is deprecated, use `iteration_range` or model slicing instead.


In [ ]:
# TRAIN ON WESAD AND TEST ON ASCERTAIN
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_w)
importlib.reload(dt)


for i, phases_wesad in enumerate(model_phases_wesad):
        print(f"WESAD PHASES {i} " + "-"*50)
        x_a, y_a = train.Train_WESAD.get_wesad_data(metrics, phases_wesad, verbose=False, label_type=wesad_label_type, threshold=threshold)
        x_b, y_b = train.Train_ASCERTAIN.get_ascertain_data(metrics, verbose=False, label_type=asc_label_type, threshold=asc_threshold)
        # drop subjects with noisy data
        inds = pd.isnull(x_b).any(axis=1).to_numpy().nonzero()[0]
        x_b = x_b.drop(labels=inds, axis=0)
        y_b = y_b.drop(labels=inds, axis=0)
        
        x_a = x_a.drop(["phaseId"], axis=1)
        x_b = x_b.drop(["phaseId"], axis=1)

        # make sure subjects from different datasets aren't labeled with the same index
        x_b["subject"] = x_b["subject"] + 500
        y_b["subject"] = y_b["subject"] + 500

        # 0-1 scaling
        for c in range(3, len(x_a.columns)):
            data_col = x_a[x_a.columns[c]]
            data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
            x_a[x_a.columns[c]] = data_col
        # 0-1 scaling
        for c in range(3, len(x_b.columns)):
            data_col = x_b[x_b.columns[c]]
            data_col = (data_col - data_col.min())/(data_col.max() - data_col.min())
            x_b[x_b.columns[c]] = data_col

        print(f"y_a:\n{y_a.loc[:, 'label'].value_counts()}")
        print(f"y_b:\n{y_b.loc[:, 'label'].value_counts()}")

        acc_results = {
        "SVM": [],
        "LogReg": [],
        "XGB": []
        }
        reports = {
        "SVM": [],
        "LogReg": [],
        "XGB": []
        }
        num_iters = 10
        for _ in range(num_iters):
            out = train.Train_Multi_Dataset.train_across_datasets(models, x_a, y_a, x_b, y_b, by_subject=True, save_metrics=True, test_size=test_size, is_resample=True)
            for model_name in acc_results:
                acc_results[model_name].append(out[model_name][0])
                reports[model_name].append(out[model_name][1])

        for model_name in acc_results.keys():
            acc = np.mean(acc_results[model_name])
            print(f"{model_name} accuracy over {num_iters} rounds: {acc}")
            if acc > 0.6:
                p = np.mean([report["precision"] for report in reports[model_name]])
                r = np.mean([report["recall"] for report in reports[model_name]])
                f1 = np.mean([report["f1"] for report in reports[model_name]])
                auc = np.mean([report["auc"] for report in reports[model_name]])
                report = reports[model_name]
                print(f"Report:\nPrecision: {p}\nRecall: {r}\nF1-score: {f1}\nAUC score: {auc}")
        print("\n")